In [9]:
# Installing packages
!pip install gurobipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
#Importing packages for the second approach: package - gurobi

from itertools import product
from math import sqrt
import itertools

import gurobipy as gp
from gurobipy import *
import pandas as pd



In [11]:
# Loading the list of capacity, fixed cost, variable cost, and demand into data frames
dmd = {'Demand': {'A': 145.4,
  'B': 84.1,
  'C': 156.4,
  'D': 1676.8,
  'E': 2719.6}}

var_co = {'A': {'A': 8, 'B': 14, 'C': 21, 'D': 21, 'E': 12},
 'B': {'A': 14, 'B': 6, 'C': 13, 'D': 14, 'E': 13},
 'C': {'A': 23, 'B': 13, 'C': 8, 'D': 10, 'E': 22},
 'D': {'A': 21, 'B': 14, 'C': 9, 'D': 3, 'E': 20},
 'E': {'A': 12, 'B': 13, 'C': 17, 'D': 20, 'E': 6}}


fix_co = {'High_C': {'A': 4730,
  'B': 7270,
  'C': 3080,
  'D': 9100,
  'E': 9500},
 'Low_C': {'A': 3230,
  'B': 4980,
  'C': 2110,
  'D': 6230,
  'E': 6500}}

cap = {'High_C': {'A': 1500,
  'B': 1500,
  'C': 1500,
  'D': 1500,
  'E': 1500},
 'Low_C': {'A': 500,
  'B': 500,
  'C': 500,
  'D': 500,
  'E': 500}}

demand = pd.DataFrame(dmd)
var_cost = pd.DataFrame(var_co)
fix_cost = pd.DataFrame(fix_co)
capacity = pd.DataFrame(cap)


In [12]:
# Initialize the model
m = gp.Model('facility_location')

# Defining decision variables
loc = ['A', 'B', 'C', 'D', 'E']
size = ['Low_C','High_C']
x = m.addVars(loc, size, vtype=GRB.BINARY) # (x[i,s])
y = m.addVars(loc, loc, vtype=GRB.CONTINUOUS) #(y[i,j])

In [14]:
# Define objective function
# MIP Model Formulation

objective = gp.quicksum(fix_cost[s][i] * x[(i, s)] for s in size for i in loc) + gp.quicksum(var_cost[i][j] * y[(i, j)]for i in loc for j in loc)
m.setObjective(objective, GRB.MINIMIZE)

In [15]:
# Defining Constraints
for j in loc:
  m.addConstr(gp.quicksum([y[(i, j)] for i in loc]) == demand.loc[j,'Demand'])
  
for i in loc:
  m.addConstr(gp.quicksum([y[(i, j)] for j in loc]) <= gp.quicksum([capacity[s][i] * x[(i,s)] for s in size]))

In [16]:
# Optimized cost - 58850.9
m.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 10 rows, 35 columns and 60 nonzeros
Model fingerprint: 0xa8d55b1a
Variable types: 25 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [3e+00, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e+01, 3e+03]
Presolve time: 0.00s
Presolved: 10 rows, 35 columns, 60 nonzeros
Variable types: 25 continuous, 10 integer (10 binary)
Found heuristic solution: objective 71734.600000

Root relaxation: objective 5.681828e+04, 6 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 56818.2753    0    3 71734.6000 56818.2753  20.8%     -    0s
H    0     0                    63242.200000 56818.2753  10.2%     -    0s
     0  